We will working with some California Census Data .We will be trying to use various 
features of an individuals to predict the what class of income they belong in (>50k or <=50k)

# The Data

#### Read the census_data.csv data with pandas

In [42]:
import pandas as pd

In [43]:
census=pd.read_csv('census_data.csv')

In [44]:
census.head()

,age,work_class,education,education_num,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,fifty_k
0,47,Private,10th,6,Divorced,Exec-managerial,Not-in-family,Amer-Indian-Eskimo,Female,45,United-States,<=50K
1,40,Private,10th,6,Divorced,Craft-repair,Not-in-family,Amer-Indian-Eskimo,Male,40,United-States,<=50K
2,30,Private,10th,6,Divorced,Other-service,Not-in-family,Amer-Indian-Eskimo,Male,40,United-States,<=50K
3,27,Private,10th,6,Divorced,Transport-moving,Not-in-family,Amer-Indian-Eskimo,Male,75,United-States,<=50K
4,38,Private,10th,6,Divorced,Machine-op-inspct,Not-in-family,Asian-Pac-Islander,Female,40,Portugal,<=50K


Tensorflow won't not understand String as labels,you will need to use pandas .apply() method to apply
a custom function that converts them to 0s and 1s. this might be hard if you are't very familier with pandas so feel free to take a peek at the the solutions for this part

Convert the Labels columns to 0's and 1s instead of string

In [45]:
census['fifty_k'].unique()

array(['<=50K', '>50K'], dtype=object)

In [46]:
def label_fix(label):
    if label =='<=50k':
        return 0
    else:
        return 1

In [47]:
census['fifty_k']=census['fifty_k'].apply(label_fix)

In [48]:
census.head()

,age,work_class,education,education_num,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,fifty_k
0,47,Private,10th,6,Divorced,Exec-managerial,Not-in-family,Amer-Indian-Eskimo,Female,45,United-States,1
1,40,Private,10th,6,Divorced,Craft-repair,Not-in-family,Amer-Indian-Eskimo,Male,40,United-States,1
2,30,Private,10th,6,Divorced,Other-service,Not-in-family,Amer-Indian-Eskimo,Male,40,United-States,1
3,27,Private,10th,6,Divorced,Transport-moving,Not-in-family,Amer-Indian-Eskimo,Male,75,United-States,1
4,38,Private,10th,6,Divorced,Machine-op-inspct,Not-in-family,Asian-Pac-Islander,Female,40,Portugal,1


## Perform a Train Test Split on the Data

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
x_data=census.drop('fifty_k',axis=1)

In [51]:
y_labels=census['fifty_k']

In [52]:
x_train,x_test,y_train,y_test=train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

## Create  the feature columns for tf.estimator

### Take note of categorical vs continious values

In [53]:
census.columns

Index(['age', 'work_class', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'sex', 'hours_per_week',
       'native_country', 'fifty_k'],
      dtype='object')

## Import Tensorflow

In [54]:
import tensorflow as tf

### create the tf.feature_column for categorical values use vocabulary lists or just use hash bucket

In [55]:

gender=tf.feature_column.categorical_column_with_vocabulary_list('sex',['Female','Male'])

In [56]:
occupation=tf.feature_column.categorical_column_with_hash_bucket('occupation',hash_bucket_size=1000)

In [57]:
matrial_status=tf.feature_column.categorical_column_with_hash_bucket('marital_status',hash_bucket_size=1000)
relationship=tf.feature_column.categorical_column_with_hash_bucket('relationship',hash_bucket_size=1000)
education=tf.feature_column.categorical_column_with_hash_bucket('education',hash_bucket_size=1000)
workclass=tf.feature_column.categorical_column_with_hash_bucket('work_class',hash_bucket_size=1000)
native_country=tf.feature_column.categorical_column_with_hash_bucket('native_country',hash_bucket_size=1000)

### Create the continious features_columns for the continious values using numeric_column

In [58]:
age=tf.feature_column.numeric_column('age')
education_num=tf.feature_column.numeric_column('education_num')
#capital_gain=tf.feature_column.numeric_column('capital_gain')
#capital_loss=tf.feature_column.numeric_column('capital_loss')
hour_per_week=tf.feature_column.numeric_column('hours_per_week')

### put all these variable into a single list with the variable name feat_cols

In [59]:
feat_cols=[gender,occupation,matrial_status,relationship,education,workclass,native_country,
            age,education_num,hour_per_week]

# Create a Input function

### Batch_size is up to you but do make sure to shuffle

In [60]:
input_func=tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,
                                              batch_size=10,
                                              num_epochs=1000,
                                              shuffle=True)

## Create your model with tf.estimator

### Create a linearClassifier(if you want to use a DNNClassifier keep in mind you will need to create embedded columns out of the cateogrical feature that use string check out the previous lecture on this for more info)

In [61]:
model=tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\satya\\AppData\\Local\\Temp\\tmpuwopxq_e', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D1FE4F3B08>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Train your model on the data for atleast 5000 steps

In [62]:
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\satya\AppData\Local\Temp\tmpuwopxq_e\model.ckpt.
INFO:tensorflow:loss = 6.931472, step = 1
INFO:tensorflow:global_step/sec: 51.3854
INFO:tensorflow:loss = 4.3280193e-05, step = 101 (2.350 sec)
INFO:tensorflow:global_step/sec: 85.3125
INFO:tensorflow:loss = 4.782111e-05, step = 201 (0.766 sec)
INFO:tensorflow:global_step/sec: 133.782
INFO:tensorflow:loss = 2.6100404e-06, step = 301 (0.744 sec)
INFO:tensorflow:global_step/sec: 132.596
INFO:tensorflow:loss = 2.5253956e-07, step = 401 (0.754 sec)
INFO:tensorflow:global_step/sec: 132.314
INFO:tensorflow:loss = 4.369076e-06, step = 501 (0.756 sec)
INFO:tensorflow:global_step/sec: 131.294
INFO:tensorflow:loss = 1.9949218e-07, step = 601 (0.762 sec)
INFO:te

# Evaluation

### Create aprediction input function Remember to only support x_test data and keep shuffle=False

In [64]:
pred_fn=tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=len(x_test),shuffle=False)

### Use model.predict() and pass in your input function.this will produce a generator of predictions which you can then transform into a list,with list()

In [65]:
pred_gn=model.predict(input_fn=pred_fn)

In [67]:
predictions=list(pred_gn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\satya\AppData\Local\Temp\tmpuwopxq_e\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [68]:
final_preds=[pred['class_ids'][0] for pred in predictions]

### Each item in your list will look like this

In [ ]:
#final_preds

### Create a list of only the class_ids key value from the prediction list of dictionaries these are the predictions you will use to compare against the real y_test values

### Import classification report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data

In [69]:
from sklearn.metrics import classification_report

In [70]:
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      9049

    accuracy                           1.00      9049
   macro avg       1.00      1.00      1.00      9049
weighted avg       1.00      1.00      1.00      9049

